In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import joblib

In [169]:
pd.pandas.set_option('display.max_columns', None) #habilitamos despliegue maximo de columnas

In [170]:
data = pd.read_csv('train.csv')
print(data.shape)


(1460, 81)


In [171]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [172]:
#separamos data para entrenamiento y prueba,
X_train, X_test, y_train, y_test = train_test_split(
                        data.drop(['Id', 'SalePrice'], axis=1),
                        data['SalePrice'],
                        test_size=0.15,
                        random_state=2021)

In [173]:
X_train.shape, X_test.shape

((1241, 79), (219, 79))

### 1. Transformación del Target a Gausssiano

In [174]:
y_train = np.log(y_train)
y_test = np.log(y_test)

### 2. Missing Values

##### 2.1 Missing Values para Variables Categóricas

In [175]:
cat_vars = [var for var in data.columns if data[var].dtype == 'O']
cat_vars = cat_vars + ['MSSubClass']

In [176]:
X_train[cat_vars] = X_train[cat_vars].astype('O')
X_test[cat_vars] = X_test[cat_vars].astype('O')

In [177]:
len(cat_vars)

44

##### 2.1.1 Detección de NaN en variables categórias

In [178]:
cat_vars_with_na = [var for var in cat_vars 
                        if X_train[var].isnull().sum() > 0]

In [179]:
cat_vars_with_na

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [180]:
X_train[cat_vars_with_na].isnull().mean().sort_values(ascending = False)

PoolQC          0.995165
MiscFeature     0.963739
Alley           0.941176
Fence           0.796938
FireplaceQu     0.477840
GarageCond      0.056406
GarageQual      0.056406
GarageFinish    0.056406
GarageType      0.056406
BsmtExposure    0.025786
BsmtFinType2    0.024980
BsmtFinType1    0.024980
BsmtCond        0.024980
BsmtQual        0.024980
MasVnrType      0.005641
Electrical      0.000806
dtype: float64

In [181]:
#variables tratadas con etiqueta de faltante por cantidad masiva de faltantes.
vars_with_missing_string = [var for var in cat_vars_with_na
                               if X_train[var].isnull().mean() > 0.2]

#variables tratadas con procedimiento por candiad adecuada de faltantes.
vars_freq_category = [var for var in cat_vars_with_na
                               if X_train[var].isnull().mean() <= 0.2]

In [182]:
vars_with_missing_string

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [183]:
vars_freq_category

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

##### Aplicamos cirterio para tratar data faltante

In [184]:
#Faltantes con etiqueta missing
X_train[vars_with_missing_string] = X_train[vars_with_missing_string].fillna('Missing')
X_test[vars_with_missing_string] = X_test[vars_with_missing_string].fillna('Missing')

In [185]:
for var in vars_freq_category:
    mode = X_train[var].mode()[0]
    
    X_train[var].fillna(mode, inplace=True)
    X_test[var].fillna(mode, inplace=True)
    
    print(var, "-------", mode)

MasVnrType ------- None
BsmtQual ------- TA
BsmtCond ------- TA
BsmtExposure ------- No
BsmtFinType1 ------- Unf
BsmtFinType2 ------- Unf
Electrical ------- SBrkr
GarageType ------- Attchd
GarageFinish ------- Unf
GarageQual ------- TA
GarageCond ------- TA


In [186]:
X_train['BsmtFinType2'].mode()[0]

'Unf'

In [187]:
X_train[cat_vars_with_na].isnull().mean().sort_values(ascending = False)

MiscFeature     0.0
Fence           0.0
PoolQC          0.0
GarageCond      0.0
GarageQual      0.0
GarageFinish    0.0
GarageType      0.0
FireplaceQu     0.0
Electrical      0.0
BsmtFinType2    0.0
BsmtFinType1    0.0
BsmtExposure    0.0
BsmtCond        0.0
BsmtQual        0.0
MasVnrType      0.0
Alley           0.0
dtype: float64

In [188]:
cat_vars_with_na = [var for var in cat_vars 
                        if X_train[var].isnull().sum() > 0]
cat_vars_with_na

[]

##### 2.2 Missing Values para Variables Numéricas

In [189]:
num_vars = [var for var in X_train.columns
               if var not in cat_vars and var != 'SalePrice']

In [190]:
len(num_vars)

35

In [191]:
##Numéricas con faltantes
num_vars_with_na = [var for var in num_vars
                       if X_train[var].isnull().sum() > 0]



In [192]:
X_train[num_vars_with_na].isnull().mean()

LotFrontage    0.179694
MasVnrArea     0.005641
GarageYrBlt    0.056406
dtype: float64

##### Aplicamos cirterio para tratar data faltante

In [193]:
for var in num_vars_with_na:
    mean_val = X_train[var].mean()
    
    print(var, mean_val)
    
    X_train[var].fillna(mean_val, inplace=True)
    X_test[var].fillna(mean_val, inplace=True)
    

LotFrontage 70.10412573673871
MasVnrArea 104.78768233387358
GarageYrBlt 1978.6046114432108


In [194]:
X_train[num_vars_with_na].isnull().mean()

LotFrontage    0.0
MasVnrArea     0.0
GarageYrBlt    0.0
dtype: float64

### 3. Variables Temporales

In [195]:
def elapsed_time(df, var, ref):
    df[var] = df[ref] - df[var]
    return df

In [196]:
year_vars = [var for var in num_vars if (('Yr' in var) or ('Year' in var))]
year_vars

['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']

In [197]:
ref = 'YrSold'

for var in year_vars:
    if(var is not ref):
        X_train = elapsed_time(X_train, var, ref)
        X_test = elapsed_time(X_test, var, ref)

In [198]:
#Eliminamos columna YrSold debido a que guardará el valor 0
X_train.drop(['YrSold'], axis=1, inplace=True)
X_test.drop(['YrSold'], axis=1, inplace=True)

### 4. Transformación de Variables Numéricas

In [199]:
#Aplicamos transformación logaritmica para variables LotFrontage, 1stFlrSF y GrLivArea
for var in ['LotFrontage', '1stFlrSF' , 'GrLivArea']:
    X_train[var] = np.log(X_train[var])
    X_test[var] = np.log(X_test[var])

In [200]:
#aplicamos transoformación de Yeo-Jonhson
X_train['LotArea'], param = stats.yeojohnson(X_train['LotArea'])

In [201]:
X_test['LotArea'] = stats.yeojohnson(X_test['LotArea'], lmbda=param)

In [202]:
print(param)

0.014359078550954491


#### 4.1 Binarización de Variables con Sesgo fuerte

In [203]:
sesgadas = ['BsmtFinSF2', 'LowQualFinSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'MiscVal']

In [204]:
for var in sesgadas:
    X_train[var] = np.where(X_train[var] == 0, 0, 1)
    X_test[var] = np.where(X_test[var] == 0, 0, 1)

### 5. Codificación de Variables Categóricas

In [205]:
#mapeo ordinal de categorías de calidadç

quality_mapping = {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5, 'Missing':0, 'NA':0, 'NaN':0}

qual_vars = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 
            'FireplaceQu', 'GarageQual', 'GarageCond']

for var in qual_vars:
    X_train[var] = X_train[var].map(quality_mapping)
    X_test[var] = X_test[var].map(quality_mapping)

In [206]:
exposure_mapping = {'No':1, 'Mn':2, 'Av':3, 'Gd':4}

var = 'BsmtExposure'

X_train[var] = X_train[var].map(exposure_mapping)
X_test[var] = X_test[var].map(exposure_mapping)

In [207]:
finish_mapping = {'Unf':1, 'LwQ': 2, 'Rec': 3, 'BLQ':4, 'ALQ':5, 'GLQ': 6}

finish_vars = ['BsmtFinType1', 'BsmtFinType2']

for var in finish_vars:
    X_train[var] = X_train[var].map(finish_mapping)
    X_test[var] = X_test[var].map(finish_mapping)

In [208]:
fence_mapping = {'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4, 'Missing':0, 'NA':0}

X_train['Fence'] = X_train['Fence'].map(fence_mapping)
X_test['Fence'] = X_test['Fence'].map(fence_mapping)

#### 5.1 Codificación de Raras (baja frecuencia)

In [209]:
qual_vars = qual_vars + finish_vars + ['Fence', 'BsmtExposure', 'GaregeFinish']

In [210]:
other_cat = [var for var in cat_vars if var not in qual_vars]

In [211]:
other_cat

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'Foundation',
 'Heating',
 'CentralAir',
 'Electrical',
 'Functional',
 'GarageType',
 'GarageFinish',
 'PavedDrive',
 'PoolQC',
 'MiscFeature',
 'SaleType',
 'SaleCondition',
 'MSSubClass']

In [212]:
len(other_cat)

31

In [213]:
#funcion para encontar etiquetas raras
def find_rare_labels_freq(df, var, rare_perc):
    temp = df.groupby(var)[var].count()/len(df)
    return temp[temp > rare_perc].index

In [214]:
for var in other_cat:
    freqValue = find_rare_labels_freq(X_train, var, 0.01)
    
    #print(var, '--------', freqValue)
    
    X_train[var] = np.where(X_train[var].isin(freqValue), X_train[var], 'Rare')
    X_test[var] = np.where(X_test[var].isin(freqValue), X_test[var], 'Rare')

In [215]:
def replace_category_vals(train, test, y_train, var, target):
    
    temp = pd.concat([train, y_train], axis=1) 
    order_labels = temp.groupby([var])[target].mean().sort_values().index
    
    ordinal_values = {k: i for i, k in enumerate(order_labels, 0)}
    
    print(var, ordinal_values)
    
    train[var] = train[var].map(ordinal_values)
    test[var] = test[var].map(ordinal_values)
    

In [216]:
for var in other_cat:
    replace_category_vals(X_train, X_test, y_train, var, 'SalePrice')

MSZoning {'Rare': 0, 'RM': 1, 'RH': 2, 'RL': 3, 'FV': 4}
Street {'Rare': 0, 'Pave': 1}
Alley {'Grvl': 0, 'Pave': 1, 'Missing': 2}
LotShape {'Reg': 0, 'IR1': 1, 'Rare': 2, 'IR2': 3}
LandContour {'Bnk': 0, 'Lvl': 1, 'Low': 2, 'HLS': 3}
Utilities {'Rare': 0, 'AllPub': 1}
LotConfig {'Inside': 0, 'Corner': 1, 'FR2': 2, 'CulDSac': 3, 'Rare': 4}
LandSlope {'Gtl': 0, 'Mod': 1, 'Rare': 2}
Neighborhood {'IDOTRR': 0, 'MeadowV': 1, 'BrDale': 2, 'BrkSide': 3, 'OldTown': 4, 'Edwards': 5, 'Sawyer': 6, 'SWISU': 7, 'NAmes': 8, 'Mitchel': 9, 'SawyerW': 10, 'NWAmes': 11, 'Rare': 12, 'CollgCr': 13, 'Gilbert': 14, 'Crawfor': 15, 'ClearCr': 16, 'Somerst': 17, 'Timber': 18, 'StoneBr': 19, 'NridgHt': 20, 'NoRidge': 21}
Condition1 {'Artery': 0, 'Feedr': 1, 'Norm': 2, 'RRAn': 3, 'Rare': 4, 'PosN': 5}
Condition2 {'Rare': 0, 'Norm': 1}
BldgType {'2fmCon': 0, 'Duplex': 1, 'Twnhs': 2, '1Fam': 3, 'TwnhsE': 4}
HouseStyle {'SFoyer': 0, '1.5Fin': 1, 'Rare': 2, '1Story': 3, 'SLvl': 4, '2Story': 5}
RoofStyle {'Gable': 0,

In [221]:

def analyse_other_cats(train, y_train, var):
    temp = pd.concat([train, y_train], axis=1)

    temp.groupby(var)['SalePrice'].median().plot.bar()
    plt.title(var)
    plt.ylabel('Price')
    plt.show()

In [ ]:
for var in other_cat:
    analyse_other_cats(X_train, y_train, var)

### 6. Feature Scaling

In [224]:
scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns
)


X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns
)

In [ ]:
X_train.head()

In [228]:
#Guardamos dataset con data preparada para entrenamiento.

X_train.to_csv('preprocess_data/prep_Xtrain.csv', index=False)
X_test.to_csv('preprocess_data/prep_Xtest.csv', index=False)

y_train.to_csv('preprocess_data/prep_ytrain', index=False)
y_test.to_csv('preprocess_data/prep_ytest', index=False)

In [230]:
joblib.dump(scaler, 'preprocess_data/minmax_scaler.joblib')

['preprocess_data/minmax_scaler.joblib']

In [238]:
np.sum(X_train[X_train == 'Unf'].sum(axis=0))

0.0